In [2]:
import os
import librosa
import _pickle as cPickle
from scipy.io.wavfile import read
from pathlib import Path
import warnings
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
from sklearn import metrics

In [3]:
train_file = "..\\data\\f_list\\wav_list_train.txt"    
test_file = "..\\data\\f_list\\wav_list_test.txt"  


def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas



In [4]:
def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    
    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

In [5]:
def get_features(file_path): 
    file_paths = open(file_path,'r')
    features_normalize = []
    total_feature = []
    features = np.asarray(())
    for path in file_paths:    
        path = path.strip()
        file_name=os.path.basename(path)
        
        audio, sr1 = librosa.load(path, sr=44100)
        data=librosa.resample(audio, sr1, 16000) #use librosa to make resample (change frome 44100 to 16000).
        vector   = extract_features(data,16000)
        if features.size == 0:
            features = vector     
        else:
            features = np.vstack((features, vector)) 

        total_feature.append(features)
    return total_feature

In [6]:
# Getting features train
extracted_features_train = get_features(train_file) 
final_features_train = np.vstack(extracted_features_train)
print(final_features_train.shape)

# Getting features test
extracted_features_test = get_features(test_file) 
final_features_test = np.vstack(extracted_features_test)
print(final_features_test.shape)


(212456, 40)
 
(45302, 40)


In [7]:
def get_label(file_path): 
    file_paths = open(file_path,'r')
    features_normalize = []
    total_feature = []
    labels = []
    features = np.asarray(())    
    for path in file_paths:    
        path = path.strip()
        file_name=os.path.basename(path)

        audio, sr1 = librosa.load(path, sr=44100)
        data=librosa.resample(audio, sr1, 16000)
        vector   = extract_features(data,16000)
        if features.size == 0:
            features = vector            
        else:
            features = np.vstack((features, vector))

        file_name=os.path.basename(path)
        split_filename=file_name.split('_')
        molecule_name=split_filename[0]
        
        for r in range(features.shape[0]):
            labels.append(molecule_name)
       
    return labels

In [8]:
# Getting labels train
train_label_value = get_label(train_file)
train_final_label = np.array(train_label_value)
print(train_final_label)
print(train_final_label.shape)
print(" ")

# Getting labels test
test_label_value = get_label(test_file)
test_final_label = np.array(test_label_value)
print(test_final_label)
print(test_final_label.shape)

['S001F' 'S001F' 'S001F' ... 'S023M' 'S023M' 'S023M']
(212456,)
 
['S031F' 'S031F' 'S031F' ... 'S062F' 'S062F' 'S062F']
(45302,)


In [9]:
x_train = preprocessing.StandardScaler().fit(final_features_train).transform(final_features_train.astype(float))
y_train = train_final_label 

x_test =  preprocessing.StandardScaler().fit(final_features_test).transform(final_features_test.astype(float))
y_test =  test_final_label   

In [10]:
# make modiling
from sklearn.neighbors import KNeighborsClassifier
k = 9 
neigh = KNeighborsClassifier(n_neighbors = k).fit(x_train ,y_train)
print(neigh) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')


In [11]:
# predicting
y_pred = neigh.predict(x_test)
#print(y_pred)

In [ ]:
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(x_train)))

In [ ]:
print("Test set Accuracy: ",  metrics.accuracy_score(y_test, y_pred))